In [29]:
import os
import json
import numpy as np
import pickle 
import tqdm
import tarfile
from scipy import sparse
from collections import defaultdict
from pprint import pprint
from collections import Counter
from itertools import permutations

# Decoder

Read steam api match and create match datapoint
 
```
for each team; predict_log_proba for each hero in team

append log_proba to

        pos

h1  p   p   p   p   p
h2  p   p   p   p   p
h3  p   p   p   p   p
h4  p   p   p   p   p
h5  p   p   p   p   p
```

Create permutations for team (0 to 4, r=5). Use itertools permutations
```
c = [1,3,0,4,2] // get the log_prob by index in pos matrix
log_p = sum(c)
if log_p > best_p:
    best_p = log_p
    best_c = c

``` 



In [85]:
with open("../data/steam_api_match_example.json", "r") as f:
    match = json.load(f)
    match = match["result"]

with open("../data/logreg_clfs_all.pkl", "rb") as f:
    clfs = pickle.load(f)

# open heroes.json 
with open("../data/heroes.json", "r") as f:
    heroes = json.load(f)
    heroes_id_to_name = {hero["id"]: hero["localized_name"] for hero in heroes}

def is_valid_match(match):
    if not match["lobby_type"] in {0, 5, 6, 7}:
        print("Invalid lobby type:", match["lobby_type"])
        return False
    if match["duration"] < 60 * 20:
        print("Invalid duration:", match["duration"])
        return False
    if not match["game_mode"] in {1, 2, 16, 22}:
        print("Invalid game mode:", match["game_mode"])
        return False

    return True

def get_rank(rank: int):
    oh = np.zeros(5)
    oh[rank] = 1
    return oh

def create_match_datapoint(match):
    datapoint = {}

    players = match["players"]
    t0 = [p for p in players if p["player_slot"] < 128]
    t1 = [p for p in players if p["player_slot"] >= 128]

    for team in [t0, t1]:
        # !Ranks
        attributes = ["gold_per_min", "xp_per_min", "kills", "deaths", "assists", "last_hits", "hero_damage", "tower_damage"]
        ranks = {}
        
        for attr in attributes:
            ranks[attr] = sorted([(p["hero_id"], p[attr]) for p in team], key=lambda x: x[1], reverse=True)

        hids = [p["hero_id"] for p in team]
        team_position_proba = defaultdict(list)

        # !Create c
        for p in team:
            features = []
            hid = p["hero_id"]

            teammates = np.zeros((136))
            for team_hid in hids:
                if team_hid != hid:
                    teammates[team_hid] = 1.
            
            features.append(teammates)
            
            for rank in ranks: 
                features.append(get_rank(ranks[rank].index((hid, p[rank]))))

            x = np.concatenate(features)

            y_pred = clfs[hid].predict_log_proba(x.reshape(1, -1))

            team_position_proba[hid] = y_pred[0]

        best_log_p = -np.inf
        best_comp = None

        for comp in permutations(range(5), 5):
            comp_with_heroid = [(comp[i], heroes_id_to_name[hid], round(team_position_proba[hid][comp[i]], 2)) for i, hid in enumerate(hids)]
            log_p = np.array([team_position_proba[hid][comp[i]] for i, hid in enumerate(hids)]).sum()
            if log_p > best_log_p:
                best_log_p = log_p
                best_comp = sorted(comp_with_heroid, key=lambda x: x[0])
            
        print("{} => {:.2f}".format(best_comp, best_log_p))
        
        
    return datapoint


if is_valid_match(match):
    match_datapoint = create_match_datapoint(match)
    # print(match_datapoint)



[(0, 'Clinkz', -0.04), (1, 'Windranger', -0.9), (2, 'Pudge', -0.39), (3, 'Zeus', -0.88), (4, 'Grimstroke', -0.12)] => -2.33
[(0, 'Sniper', -1.09), (1, 'Void Spirit', -0.01), (2, 'Underlord', -0.01), (3, 'Earthshaker', -0.1), (4, 'Keeper of the Light', -0.27)] => -1.48
